In [1]:
import h2o
import time,json,os
import numpy as np
import pandas as pd
from jsvm import CJSVM
from common import preprocess,get_csv_files
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import fbeta_score
from warnings import filterwarnings
filterwarnings("ignore") 
np.set_printoptions(suppress=True)
pd.set_option('display.float_format',lambda x : '%.8f' % x)

In [2]:
#scan path and find all trinned models files
def get_model_files(filepath,file_list=[]):
    files = os.listdir(filepath)
    for fi in files:
        fi_d = os.path.join(filepath,fi)
        if os.path.isdir(fi_d):
            get_model_files(fi_d,file_list)
        else:
            if fi_d.find("_model_python_") > 0:
                file_list.append(fi_d)
                
#get all trained model information
def get_all_models():
    model_list = []
    model_file_list = []
    model_path = "/data/dataset/model"
    get_model_files(model_path , model_file_list)
    for fi in model_file_list:
        t = fi.split("/")
        model_file = fi
        model_name = t[-2]
        dataset =  t[-4]
        train_set = "%s/dataset/%s/train.csv"%(model_path,dataset)
        valid_set = "%s/dataset/%s/valid.csv"%(model_path,dataset)
        temp = {}
        temp['model_file'] = model_file
        temp['model_name'] = model_name
        temp['dataset'] = dataset
        temp['train_set'] = train_set
        temp['valid_set'] = valid_set
        model_list.append(temp)
    return pd.DataFrame(model_list)

In [3]:
class CJPerformance(object):
    
    def __init__(self,model_file):
        self.m_model_file = model_file
        if not self.is_svm():
            h2o.init(ip="localhost" , port=54321)
            self.m_model = h2o.load_model(model_file)
        else:
            self.m_model = CJSVM()
            self.m_model.load(model_file)
    
    def is_svm(self):
        if self.m_model_file.find("sklearn_model_python_svm.pkl") >= 0:
            return True
        else:
            return False
    
    def predict(self , sample_file , need_preprocess = True ):
        df = pd.read_csv(sample_file,index_col=0)
        if need_preprocess:
            df_all = preprocess(df)
        else:
            df_all = df.copy( deep = True )
        if not self.is_svm():
            df_h2o = h2o.H2OFrame(df_all)
            x = df_h2o.columns
            y = "label"
            #x.remove(y)
            df_h2o[y] = df_h2o[y].asfactor()
            pred = self.m_model.predict(df_h2o).as_data_frame()
            y_true = df_all[y].to_list()
            y_pred = pred['predict']
        else:
            X = df_all.copy( deep = True )
            Y = X['label']
            del X['label']
            y_true = df_all['label'].to_list()
            y_pred = self.m_model.predict(X)
            
        return y_true,y_pred
    
    def Test(self , sample_file , need_preprocess = True):
        y_true , y_pred = self.predict( sample_file , need_preprocess )
        result = {}
        result['confusion_matrix'] = confusion_matrix(y_true,y_pred)
        result['roc_curve'] = roc_curve(y_true,y_pred)
        result['recall'] = recall_score(y_true,y_pred)
        result['mcc'] = matthews_corrcoef(y_true,y_pred)
        result['accuracy'] = accuracy_score(y_true,y_pred)
        result['precision'] = precision_score(y_true,y_pred)
        result['auc'] = roc_auc_score(y_true,y_pred)
        result['log_loss'] = log_loss(y_true,y_pred)
        result['f1_score'] = f1_score(y_true,y_pred)
        result['fbeta_score'] = fbeta_score(y_true,y_pred,beta=0.5)
        return pd.Series(result)

In [4]:
csv_list = []
get_csv_files("/data/dataset/sample/",csv_list)
csv_list.sort()
all_models = get_all_models()
all_result = []
for item in json.loads(all_models.to_json(orient="records")):
    model_name = item['model_name']
    dataset = item['dataset']
    model_file = item['model_file']
    if model_file.find("PSVM_model_python") >= 0:
        continue
    perf = CJPerformance(model_file)
    for csv in csv_list:
        valid_set = os.path.splitext(os.path.basename(csv))[0]
        print(dataset, model_name , valid_set)
        result = perf.Test(csv,True)
        result['model'] = model_name
        result['train'] = dataset
        result['valid'] = valid_set
        all_result.append(result)
df_result = pd.DataFrame(all_result)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,58 mins 23 secs
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 18 days
H2O_cluster_name:,root
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.46 Gb
H2O_cluster_total_cores:,22
H2O_cluster_allowed_cores:,22
H2O_cluster_status:,"locked, healthy"


xgboost base-3-2-test-1
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost base-0-1-test-1
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost base-1-1-test-2
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost base-3-1-test-2
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost base-0-1-test-2
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progre

H2O_cluster_uptime:,1 hour 3 mins
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 18 days
H2O_cluster_name:,root
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.62 Gb
H2O_cluster_total_cores:,22
H2O_cluster_allowed_cores:,22
H2O_cluster_status:,"locked, healthy"


deeplearn base-3-2-test-1
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearn base-0-1-test-1
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearn base-1-1-test-2
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearn base-3-1-test-2
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearn base-0-1-test-2
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning pr

...

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:  8.9min remaining: 62.0min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed: 10.5min finished


svm base-0-1-train-1


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:  3.8min remaining: 26.6min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:  5.0min finished


svm base-3-1-test-1


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   46.6s remaining:  5.4min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:  1.1min finished


In [5]:
df_result.to_csv("/data/dataset/analyse/validation.csv")

In [6]:
df_result

,confusion_matrix,roc_curve,recall,mcc,accuracy,precision,auc,log_loss,f1_score,fbeta_score,model,train,valid
0,"[[27470, 5855], [4529, 12146]]","([0.0, 0.1756939234808702, 1.0], [0.0, 0.72839...",0.72839580,0.54286572,0.79232000,0.67474029,0.77635094,7.17310671,0.70054216,0.68482955,xgboost,base-3-2-train-1,base-3-2-test-1
1,"[[28581, 1395], [15159, 969]]","([0.0, 0.04653722978382706, 1.0], [0.0, 0.0600...",0.06008185,0.02928739,0.64094222,0.40989848,0.50677231,12.40144065,0.10480208,0.18937617,xgboost,base-3-2-train-1,base-0-1-test-1
2,"[[7785, 16081], [150372, 75762]]","([0.0, 0.673803737534568, 1.0], [0.0, 0.335031...",0.33503144,-0.20649683,0.33418800,0.82490772,0.33061385,22.99638322,0.47652503,0.63825808,xgboost,base-3-2-train-1,base-1-1-test-2
3,"[[41434, 82841], [42548, 83177]]","([0.0, 0.6665942466304566, 1.0], [0.0, 0.66157...",0.66157884,-0.00530931,0.49844400,0.50101194,0.49749230,17.32339549,0.57020734,0.52657202,xgboost,base-3-2-train-1,base-3-1-test-2
4,"[[131981, 18169], [73698, 6674]]","([0.0, 0.121005661005661, 1.0], [0.0, 0.083038...",0.08303887,-0.05834746,0.60148272,0.26864710,0.48101660,13.76436218,0.12686404,0.18565293,xgboost,base-3-2-train-1,base-0-1-test-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,"[[6883, 241389], [5682, 246046]]","([0.0, 0.9722763742991558, 1.0], [0.0, 0.97742...",0.97742802,0.01645644,0.50585800,0.50477705,0.50257582,17.06744607,0.66574220,0.55882257,svm,base-1-1-train-1,base-3-1-test-3
444,"[[4820, 161160], [10507, 73513]]","([0.0, 0.9709603566694782, 1.0], [0.0, 0.87494...",0.87494644,-0.18905722,0.31333200,0.31325717,0.45199304,23.71718796,0.46134054,0.35940226,svm,base-1-1-train-1,base-3-2-train-1
445,"[[8889, 322687], [20973, 147451]]","([0.0, 0.9731916664656067, 1.0], [0.0, 0.87547...",0.87547499,-0.19489024,0.31268000,0.31363344,0.45114166,23.73970783,0.46182203,0.35981632,svm,base-1-1-train-1,base-3-2-test-3
446,"[[139826, 9958], [14450, 66288]]","([0.0, 0.06648240132457406, 1.0], [0.0, 0.8210...",0.82102603,0.76506502,0.89411857,0.86939643,0.87727182,3.65704972,0.84451919,0.85927170,svm,base-1-1-train-1,base-0-1-train-1
